In [1]:
import pandas as pd
import numpy as np
import chromadb 
import tensorflow as tf
import re

2024-09-14 17:59:59.709381: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-14 17:59:59.740123: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 17:59:59.740154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 17:59:59.740937: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-14 17:59:59.745805: I tensorflow/core/platform/cpu_feature_guar

In [2]:
client = chromadb.Client()
collection = client.get_or_create_collection("height")

data = pd.read_csv("data/height_with_ocr.csv")
units = data['unit']
string_lookup = tf.keras.layers.StringLookup()
string_lookup.adapt(units)

2024-09-14 18:00:01.662291: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-14 18:00:01.719922: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-14 18:00:01.720064: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
def remove_numbers(text):
    return re.sub(r'\d+', '', text)
                  
values = string_lookup.get_vocabulary()
index = 0
for val in values:
    print(val)
    if val != "[UNK]":
        matching_rows = data[data['unit'] == val]
        
        # Select up to 10 values from the filtered dataset
        n = 200
        if len(matching_rows) <= 200:
            n = len(matching_rows)
        selected_rows = matching_rows.sample(n=n, random_state=42) if len(matching_rows) >= 100 else matching_rows
        
        for _, row in selected_rows.iterrows():
            ocr_text = row['ocr_text']
            ocr_text = remove_numbers(ocr_text)
            # Generate a random UUID for each document
            index += 1 
            
            # Add to the vector database
            collection.add(
                documents = [ocr_text], 
                metadatas=[{"units": val}],
                ids=[str(index)]
            )
print("added data into dataset")

[UNK]
centimetre
inch
millimetre
metre
foot
added data into dataset


In [4]:
ocr_text = data['ocr_text']
labels = data['unit']

In [5]:
from collections import Counter

def query_and_find_most_frequent(query_text):
    # query_text = remove_numbers(query_text)
    results = collection.query(query_texts=[query_text],
                               where = {"entity_"}
                               n_results=5)
    
    string_values = [result['units'] for result in results['metadatas'][0]]
    
    string_counts = Counter(string_values)
    
    most_frequent_string = string_counts.most_common(1)[0][0]
    
    return most_frequent_string

In [6]:
import concurrent.futures

# Function to process each item
def process_item(idx, text, label):
    res = query_and_find_most_frequent(text)
    return idx, res == label

# Initialize variables
total_count = len(labels)
count = 0

# Use ThreadPoolExecutor to run the loop in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit tasks to the executor
    futures = [executor.submit(process_item, idx, text, labels[idx]) for idx, text in enumerate(ocr_text)]
    
    # Process the results as they complete
    for future in concurrent.futures.as_completed(futures):
        idx, is_correct = future.result()
        if idx % 1000 == 0 and idx != 0:
            print("At", idx, "Accuracy:", str(count / idx))
        if is_correct:
            count += 1

# Print the final accuracy
print(count / total_count)
    

At 1000 Accuracy: 0.573
At 2000 Accuracy: 0.5915
At 3000 Accuracy: 0.592
At 4000 Accuracy: 0.58775


In [ ]:
for idx, val in enumerate(ocr_text):
    continue
print("done")

In [ ]:
text = """
134cm 182cm 211cm 10Ocm 216cm 108cm
"""
res = query_and_find_most_frequent(text)
print(res)

In [ ]:
for result in res['metadatas'][0]:
    print(result['units'])